In [11]:
import pandas as pd

In [31]:
file_paths = {
    "wind_solar_DK1": "../data/Generation Forecasts for Wind and Solar_DK1.csv",
    "wind_solar_DK2": "../data/Generation Forecasts for Wind and Solar_DK2.csv",
    "imbalance_DK1": "../data/Imbalance_year_DK1.csv",
    "imbalance_DK2": "../data/Imbalance_year_DK2.csv"
}

wind_solar_DK1 = pd.read_csv(file_paths["wind_solar_DK1"])
wind_solar_DK2 = pd.read_csv(file_paths["wind_solar_DK2"])
imbalance_DK1 = pd.read_csv(file_paths["imbalance_DK1"])
imbalance_DK2 = pd.read_csv(file_paths["imbalance_DK2"])

In [32]:
def extract_datetime(df, column_name):
    df["Datetime"] = pd.to_datetime(df[column_name].str.split(" - ").str[0], format="%d.%m.%Y %H:%M")
    df.drop(columns=[column_name], inplace=True)
    return df


wind_solar_DK1 = extract_datetime(wind_solar_DK1, "MTU (CET/CEST)")
wind_solar_DK2 = extract_datetime(wind_solar_DK2, "MTU (CET/CEST)")
imbalance_DK1 = extract_datetime(imbalance_DK1, "Imbalance settlement period CET/CEST")
imbalance_DK2 = extract_datetime(imbalance_DK2, "Imbalance settlement period CET/CEST")

In [33]:
# Select only relevant columns (ignore "Current" columns)
columns_to_keep = [col for col in wind_solar_DK1.columns if "Current" not in col and col != "Datetime"]
wind_solar_DK1 = wind_solar_DK1[["Datetime"] + columns_to_keep]
wind_solar_DK2 = wind_solar_DK2[["Datetime"] + [col.replace("DK1", "DK2") for col in columns_to_keep]]

for df in [wind_solar_DK1, wind_solar_DK2, imbalance_DK1, imbalance_DK2]:
    for col in df.columns:
        if col != "Datetime":
            df[col] = pd.to_numeric(df[col], errors="coerce")

wind_solar_all = wind_solar_DK1.merge(wind_solar_DK2, on="Datetime", suffixes=("_DK1", "_DK2"))

for col in columns_to_keep:
    dk1_col = col
    dk2_col = col.replace("DK1", "DK2")
    wind_solar_all[col.replace("DK1", "All")] = wind_solar_all[dk1_col] + wind_solar_all[dk2_col]

# Keep only the final cleaned columns
wind_solar_all = wind_solar_all[["Datetime"] + [col for col in wind_solar_all.columns if "All" in col]]


In [34]:
imbalance_DK1["Total Imbalance [MWh] - IBA|DK1"] = pd.to_numeric(imbalance_DK1["Total Imbalance [MWh] - IBA|DK1"], errors="coerce")
imbalance_DK2["Total Imbalance [MWh] - IBA|DK2"] = pd.to_numeric(imbalance_DK2["Total Imbalance [MWh] - IBA|DK2"], errors="coerce")

imbalance_all = imbalance_DK1.merge(imbalance_DK2, on="Datetime")
imbalance_all["Total Imbalance [MWh] - IBA|All"] = imbalance_all["Total Imbalance [MWh] - IBA|DK1"] + imbalance_all["Total Imbalance [MWh] - IBA|DK2"]
imbalance_all = imbalance_all[["Datetime", "Total Imbalance [MWh] - IBA|All"]]

In [35]:
final_dataset = wind_solar_all.merge(imbalance_all, on="Datetime")

In [36]:
output_path = "../data/cleaned_aggregated_energy_data.csv"
final_dataset.to_csv(output_path, index=False)